In [ ]:
# Dowloading pyspark
#p!pip install pyspark

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

PREPARE THE ENVIRONMENT, UPLOAD DATA, PREPROCESS DATA AND CREATE THE TABLES: Author, Paper, Affiliation, Book, Journal and Conference

In [ ]:
# With sparkSession we create a connection to our database
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, TimestampType
from pyspark.sql.functions import count, col, xxhash64, collect_list, explode

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
      .master("local") \
      .appName("Bibliography") \
      .getOrCreate()

INPUT_FILE = "bibliography.json"
OPTIONS = {'multiline': 'true', 'allowNumericLeadingZero': 'true','timestampFormat': "yyyy-MM-dd'T'HH:mm:ss[.ZZZ'Z']"}

In [ ]:
#AUTHOR TABLE
schemaAut = StructType(
            [StructField('authors', ArrayType(StructType([
                StructField('_id', StringType(), nullable = False),
                StructField('name', StringType(), True),
                StructField('email', StringType(), True),
                StructField('bio', StringType(), True),
                ])), True)
            ])

df_aut = spark.read.format('json').options(**OPTIONS).schema(schemaAut).json(INPUT_FILE)
df_aut = df_aut.select(explode(df_aut.authors))
df_aut = df_aut.withColumnRenamed("col", "authors")
df_aut = df_aut.filter(col("authors._id") != "null").select("authors._id","authors.name","authors.email", "authors.bio")
df_aut = df_aut.withColumnRenamed("_id", "author_id")
df_aut = df_aut.dropDuplicates(["author_id"])
df_aut.printSchema()
df_aut.show()

In [ ]:
# PAPER TABLE WITHOUT PUBLICATION_ID
schemaPaper = StructType(
            [StructField('_id', StringType(), True),
             StructField('title', StringType(),True),
             StructField('keywords', ArrayType(StringType()), True),
             StructField('fos', ArrayType(StringType()), True),
             StructField('references', ArrayType(StringType()), True),
             StructField('page_start', IntegerType(), True),
             StructField('page_end', IntegerType(), True),
             StructField('lang', StringType(),True),
             StructField('abstract', StringType(),True),
             StructField('publication_type', StringType(),True),
             StructField('date', TimestampType(), True),
             StructField('doi', StringType(),True),
             StructField('url', ArrayType(StringType()),True),
            ])

df_paper = spark.read.format('json').options(**OPTIONS).schema(schemaPaper).json(INPUT_FILE)
df_paper = df_paper.withColumnRenamed("_id", "paper_id")
df_paper.printSchema()
df_paper.show()

In [ ]:
# AFFILIATION TABLE
schemaAffiliation = StructType(
            [StructField('_id', StringType(), True),
             StructField('authors', ArrayType(StructType([
                    StructField('_id', StringType(), True),
                    StructField('org', StringType(), True)
             ])), True),
            ])

df_aff = spark.read.format('json').options(**OPTIONS).schema(schemaAffiliation).json(INPUT_FILE)
df_aff = df_aff.withColumnRenamed("_id", "paper_id")
df_aff = df_aff.select("paper_id", explode(df_aff.authors))
df_aff = df_aff.withColumnRenamed("col", "authors")
df_aff = df_aff.filter(col("authors._id") != "null").filter(col("paper_id") != "null").select("paper_id", "authors._id","authors.org")
df_aff = df_aff.withColumnRenamed("_id", "author_id")
df_aff = df_aff.dropDuplicates(["author_id", "paper_id"])
df_aff = df_aff.withColumnRenamed("org", "organization")
df_aff.printSchema()
df_aff.show()

In [ ]:
# JOURNAL TABLE
# Preprocessing of the journals for cleaning and merging the journals

journal_schema_preprocessing = StructType(
    [StructField('_id', StringType(), True),
     StructField('issn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('volume', IntegerType(), True),
     StructField('issue', IntegerType(), True),
     StructField('publication_type', StringType(),True)])

# Reading the json file
df_journals_to_filter = spark.read.format('json').options(**OPTIONS).schema(journal_schema_preprocessing).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_journals_to_filter = df_journals_to_filter.filter(col('publication_type') == 'Journal').filter(col('issn') != 'null').filter(col('venue') != 'null').filter(col('issue') >= 0).filter(col('volume') >= 0)
df_journals_to_filter = df_journals_to_filter.groupBy('venue', 'volume', 'issue', 'issn').agg(collect_list('publisher').alias('publishersArray'), collect_list('_id').alias('_id'), count(col('publisher'))) # count can be removed (I was interested in evaluating if the group by was meaningful)
df_journals_to_insert = df_journals_to_filter.withColumn('publisher', df_journals_to_filter['publishersArray'][0]).select('venue', 'volume', 'issue', 'publisher', 'issn', '_id')

# Adding the new column which contains the publication_identifier
df_journals = df_journals_to_insert.withColumn("publication_id", xxhash64('venue', 'volume', 'issue', 'issn'))

# Adding the foreign key to the papers
exploded_journals = df_journals.select(explode('_id'), 'publication_id')
#exploded_journals.show(truncate = False)

df_papers_in_journals = exploded_journals.join(df_paper, exploded_journals.col == df_paper.paper_id, "inner")
df_papers_in_journals = df_papers_in_journals.drop('col')

df_journals = df_journals.drop(df_journals._id)

# Visualizing the data
# print('Papers')
# df_papers_in_journals.show(truncate = False)
print('Schema of the journals')
df_journals.printSchema()
print('Journals')
df_journals.show(truncate=False)

In [ ]:
# BOOK TABLE
# Preprocessing of the books for cleaning and merging the books
book_schema_preprocessing = StructType(
    [StructField('_id', StringType(), True),
     StructField('isbn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('publication_type', StringType(),True)])

# Reading the json file
df_books_to_filter = spark.read.format('json').options(**OPTIONS).schema(book_schema_preprocessing).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_books_to_filter = df_books_to_filter.filter(col('publication_type') == 'Book').filter(col('isbn') != 'null').filter(col('venue') != 'null')
df_books_to_filter = df_books_to_filter.groupBy('isbn', 'venue').agg(collect_list('publisher').alias('publishersArray'), collect_list('_id').alias('_id'), count(col('publisher'))) # count can be removed (I was interested in evaluating if the group by was meaningful)
dfbooks_to_insert = df_books_to_filter.withColumn('publisher', df_books_to_filter['publishersArray'][0]).select('venue', 'isbn', 'publisher', '_id')

# Adding the new column which is the id
df_books = dfbooks_to_insert.withColumn('publication_id', xxhash64('isbn', 'venue'))

# Adding the foreign key to the papers
exploded_books = df_books.select(explode('_id'), 'publication_id')
# exploded_books.show(truncate = False)

df_papers_in_books = exploded_books.join(df_paper, exploded_books.col == df_paper.paper_id)
df_papers_in_books = df_papers_in_books.drop('col')

df_books = df_books.drop(df_books._id)

# Visualizing the data
# print('Papers')
# df_papers_in_books.show(truncate = False)
print('Schema of the books')
df_books.printSchema()
print('Books')
df_books.show(truncate=False)

In [ ]:
# CONFERENCE TABLE
# Preprocessing of the books for cleaning and merging the books

schemaConf = StructType(
    [StructField('_id', StringType(), True),
     StructField('location', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('publication_type', StringType(),True)])
# Reading the json file
df_conferences_to_filter = spark.read.format('json').options(**OPTIONS).schema(schemaConf).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_conferences_to_filter = df_conferences_to_filter.filter(col('publication_type') == 'Conference').filter(col('venue') != 'null')
df_conferences_to_filter = df_conferences_to_filter.groupBy('venue').agg(collect_list('location').alias('locations_array'), collect_list('_id').alias('_id'), count(col('location'))) # count can be removed (I was interested in evaluating if the group by was meaningful)
df_conferences_to_insert = df_conferences_to_filter.withColumn('location', df_conferences_to_filter['locations_array'][0]).select('venue', 'location', '_id')

# Adding the new column which is the id
df_conferences = df_conferences_to_insert.withColumn('publication_id', xxhash64('venue'))

# Adding the foreign key to the papers
exploded_conferences = df_conferences.select(explode('_id'), 'publication_id')
#exploded_conferences.show(truncate = False)

df_papers_in_conferences = exploded_conferences.join(df_paper, exploded_conferences.col == df_paper.paper_id)
df_papers_in_conferences = df_papers_in_conferences.drop('col')

df_conferences = df_conferences.drop(df_conferences._id)

# Visualizing the data
#print('Papers')
#df_papers_in_conferences.show(truncate = False)
print('Schema of the conferences')
df_conferences.printSchema()
print('Conferences')
df_conferences.show(truncate=False)

In [ ]:
# Merging the 3 dataframe which one contains the papers published in a specific media
df_papers = df_papers_in_books.union(df_papers_in_journals).union(df_papers_in_conferences)

# Visualizing the data
print('Papers schema')
df_papers.printSchema()
print('Papers data')
df_papers.show()

In [ ]:
# For checking the result
print('Papers published in books')
df_papers.filter(col('publication_type') == 'Book').select('paper_id', 'title', 'publication_type', 'publication_id').show()
print('Papers published in journals')
df_papers.filter(col('publication_type') == 'Journal').select('paper_id', 'title', 'publication_type', 'publication_id').show()
print('Papers published in conferences')
df_papers.filter(col('publication_type') == 'Conference').select('paper_id', 'title', 'publication_type', 'publication_id').show()

COMMANDS

In [ ]:
# Command 1: Add a new row to the Paper DataFrame

new_paper_file = 'single_paper.json'

new_paper = spark.read.options(**OPTIONS).json(new_paper_file, schemaPaper)\
                 .withColumnRenamed("_id", "paper_id")

journal_schema = StructType(
    [StructField('issn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('volume', IntegerType(), True),
     StructField('issue', IntegerType(), True)])


journal = spark.read.options(**OPTIONS).json(new_paper_file, journal_schema)\
               .withColumn("publication_id", xxhash64('venue', 'volume', 'issue', 'issn'))

foreign_key = journal.head(1)[0].asDict()['publication_id']

if df_journals.filter(col('publication_id') == foreign_key).count() == 0:
    df_jounrals = df_journals.union(journal)

paper_id = new_paper.head(1)[0].asDict()['paper_id']

# Inserting foreign_key at position 0
new_paper = new_paper.select(lit(foreign_key).alias('publication_id'), '*')

if df_papers.filter(col('paper_id') == paper_id).collect() == []:
    df_papers = df_papers.union(new_paper)

In [ ]:
# Update one single row of a dataframe or multiple rows

#The command modifies the DOI and URL of a paper with identifier equal to '53e997e4b7602d9701fdb48a'.
#For doing this operation we firstly filter the dataframe keeping only the row to be modified. We add a column for each value we want to insert under the name new_field_name. Then we drop the old columns containing the previous values and we rename the new columns with the name of the old ones.
#Finally, we make the union between the entire dataframe, without the row we want to modify, and the new entry.
from pyspark.sql.functions import lit, array

#df_papers\
#    .filter(col('title').like('%chatbot%'))\
#    .select('title', 'paper_id')\
#    .show(truncate = False)
updated_df_papers = df_papers\
    .filter(col('paper_id') == '53e997e4b7602d9701fdb48a')
updated_df_papers = updated_df_papers\
    .withColumn('new_doi', lit('10.1007/11944577_37'))\
    .withColumn('new_url', array([lit('https://link.springer.com/chapter/10.1007/11944577_37')]))
updated_df_papers = updated_df_papers\
    .drop(col('doi')).drop(col('url'))\
    .withColumnRenamed('new_doi', 'doi')\
    .withColumnRenamed('new_url', 'url')

# To check the number of the paper is the same
updated_df_papers = df_papers.filter(col('paper_id') != '53e997e4b7602d9701fdb48a').union(updated_df_papers)

print('The size of the entire initial database is ' + str(df_papers.count()) + ', the size of the current database is ' + str(updated_df_papers.count()))
updated_df_papers.filter(col('paper_id') == '53e997e4b7602d9701fdb48a').select('paper_id', 'title', 'doi', 'url').show(truncate = False)

In [ ]:
# Command 4: delete a group of rows
from pyspark.sql.functions import year

df_papers = df_papers\
    .filter(year('date') > '1950')\
    .select('title', 'publication_type', 'date')\
    .orderBy('date')\
    .show(truncate = False)

In [ ]:
# Command 5: create a new column with the length of the paper (number of total pages)

df_papers_total_pages = df_papers \
    .filter((col('page_start') >= 0) & (col('page_end') >= 0) & (col('page_start') <= col('page_end'))) \
    .withColumn('total_pages', col('page_end') - col('page_start'))

df_papers_total_pages \
    .select(col('title'), col('page_start'), col('page_end'), col('total_pages')) \
    .show(5, truncate=False)

QUERIES

In [ ]:
#Query 1: WHERE, JOIN

venue, volume, issue = ('BMC Bioinformatics', '14', '1') 

df_papers_q1 = df_journals\
               .filter((col('venue') == venue) &
                       (col('volume') == volume) &
                       (col('issue') == issue))\
                .join(df_papers,
                      (df_journals['publication_id'] == df_papers['publication_id']) &
                        (df_papers['publication_type'] == 'Journal')
                     )

df_papers_q1.select(['paper_id', 'title']).show(truncate=60)

In [ ]:
# Query 2: WHERE, LIMIT, LIKE
# Find the papers written in the last twenty years in english whose keywords have the word \verb|artificial| inside the keywords. We require that these papers have the DOI set to a not null value.
# The results are ordered ascending by the date and only 20 elements are printed.

from pyspark.sql.functions import current_timestamp, unix_timestamp

df = df_papers.withColumn('current time', current_timestamp())
df = df\
    .filter(
        (((unix_timestamp('current time') - unix_timestamp('date')) / 3600 / 24 / 365) > 50) & 
        (col('doi').isNotNull()) &
        (col('lang') == 'eng'))\
    .select('title', 'date', explode('keywords').alias('keyword'))\
    .filter(col('keyword').like('%artificial%'))\
    .distinct()\
    .sort(col('date').asc())\
    .limit(20)\
    .show(truncate = False)

In [ ]:
# Query 4: GROUP BY, JOIN, AS
from pyspark.sql.functions import collect_set, size

df = df_aff\
    .join(df_papers, df_papers.paper_id == df_aff.paper_id, 'inner')\
    .drop(df_papers.paper_id).select('paper_id', 'organization', 'publication_type')\
    .filter((col('organization').isNotNull()) & (col('organization') != "") & (col('publication_type') == "Conference"))\
    .groupBy('organization')\
    .agg(collect_set('paper_id').alias('papers'))\
    .filter(size(col('papers')) > 10)\
    .show(truncate = 50)

In [ ]:
# Query 5: WHERE, GROUP BY
# Retrieve some statistics about papers
from pyspark.sql.functions import sum, min, max, avg, format_number, variance

df_papers_total_pages.filter(year(col('date')) >= 2015) \
    .groupBy(year(col('date')).alias('year')) \
    .agg(count('paper_id').alias('total_papers'),
         sum('total_pages').alias('total_pages'),
         min('total_pages').alias('min_pages'),
         max('total_pages').alias('max_pages'),
         format_number(avg('total_pages'), 2).alias('avg_pages'),
         format_number(variance('total_pages'), 2).alias('var_pages')) \
    .sort(col('year').desc()) \
    .show()

In [ ]:
# Query 6: GROUP BY, HAVING, AS

df_papers_q6 = df_papers\
               .select('paper_id',
                       'title',
                       explode(col('references')).alias('reference'))\
               .groupBy('reference')\
               .agg(count('paper_id').alias('references_count'))\
               .filter(col('references_count') > 30)\
               .join(df_papers,
                     col('reference') == df_papers.paper_id)\
               .sort(col('references_count').desc())\

df_papers_q6.select(['title', 'references_count']).show(truncate=False)

In [ ]:
# Query 7: WHERE, GROUP BY, HAVING, AS
# The query returns the association between fields of study and keywords which are more present within the database and how many times they appear together
from pyspark.sql.functions import year, col, lit
df = df_papers
df = df.withColumn('count', lit(1))
df = df\
    .filter(
        (col('doi').isNotNull()) &
        (year(col('date')) >= 2000) &
        (size(col('fos')) > 0) &
        (size(col('keywords')) > 0))\
    .select('fos', 'count', explode('keywords').alias('keyword'))\
    .select('keyword', explode('fos').alias('fos'), 'count')\
    .groupby('fos', 'keyword')\
    .sum('count')\
    .withColumnRenamed('sum(count)', 'couple count')\
    .filter(col('couple count') > 100)\
    .sort(col('couple count').desc())\
    .show(truncate = False)

In [ ]:
# Query 9: WHERE, GROUP BY, HAVING, 1 JOIN

from pyspark.sql.functions import collect_set, concat, size

df = df_journals\
    .withColumnRenamed('venue', 'venueJournals')\
    .filter((col('volume')) > 10)\
    .join(df_books,df_books.publisher == df_journals.publisher,"inner")\
    .drop(df_journals.publisher)\
    .withColumnRenamed('venue', 'venueBooks')\
    .select('venueBooks', 'venueJournals', 'publisher')\
    .dropDuplicates(['venueBooks', 'venueJournals', 'publisher'])\
    .groupBy('publisher')\
    .agg(collect_set('venueBooks').alias('books'),
         collect_set('venueJournals').alias('journals'))\
    .withColumn("total_publications_per_publisher",
                concat(col("books"), col("journals")))\
    .filter(size(col("total_publications_per_publisher")) > '500')\
    .select('publisher', "total_publications_per_publisher")\
    .show(truncate = 50)

In [ ]:
# Query 10: WHERE, GROUP BY, HAVING, 2 JOINs
# Retrieve authors who worked for at least 3 different organizations and have published at least 3 papers with at least 5 fos and 5 references each

from pyspark.sql.functions import approx_count_distinct

df_papers \
    .filter((size(col('fos')) >= 5) & (size(col('references')) >= 5)) \
    .join(df_aff, df_papers.paper_id == df_aff.paper_id, "inner") \
    .drop(df_aff.paper_id) \
    .join(df_aut, df_aff.author_id == df_aut.author_id, "inner") \
    .drop(df_aff.author_id) \
    .groupBy("author_id") \
    .agg(count("paper_id").alias("papers_count"),
         approx_count_distinct("organization").alias("organizations_count"),
         collect_set("name").alias("name")) \
    .filter((size("name") == 1) & (col("papers_count") >= 3)  & (col("organizations_count") >= 3)) \
    .orderBy(col("papers_count").desc(), col("organizations_count").desc()) \
    .select(explode(col("name")).alias("name"), col("papers_count"), col("organizations_count")) \
    .show(5)